In [1]:
using LinearAlgebra
using Random

Random.seed!(0)
A = rand(-5:5, 5, 6)
@show A

A = [-1 -4 0 4 -5 1; -5 -4 4 3 0 -1; 4 2 -3 -4 1 1; -5 -2 -2 1 -5 4; 2 -5 -1 -1 -1 0]


5×6 Matrix{Int64}:
 -1  -4   0   4  -5   1
 -5  -4   4   3   0  -1
  4   2  -3  -4   1   1
 -5  -2  -2   1  -5   4
  2  -5  -1  -1  -1   0

In [2]:
function det_without_col(M, i)
    N = M[:, 1:end .!= i]
    return Int(round(det(N)))
end

for i in 1:size(A, 2)
    println(i, " ", det_without_col(A, i))
end

1 -58
2 -88
3 180
4 337
5 -399
6 353


In [3]:
function det_without_col_replacing_col(M, i, j, col)
    N = copy(M)
    N[:, j] = col
    N = N[:, 1:end .!= i]
    return Int(round(det(N)))
end

for i in 1:size(A, 2)
    for j in 1:size(A, 2)
        if i == j
            continue
        end
        println(i, " ", j, " ", det_without_col_replacing_col(A, i, j, A[:, i]))
        println(i, " ", j, " ", det_without_col_replacing_col(A, i, j, A[:, j]))
    end
end

1 2 -88
1 2 -58
1 3 -180
1 3 -58
1 4 337
1 4 -58
1 5 399
1 5 -58
1 6 353
1 6 -58
2 1 -58
2 1 -88
2 3 180
2 3 -88
2 4 -337
2 4 -88
2 5 -399
2 5 -88
2 6 -353
2 6 -88
3 1 58
3 1 180
3 2 -88
3 2 180
3 4 337
3 4 180
3 5 399
3 5 180
3 6 353
3 6 180
4 1 -58
4 1 337
4 2 88
4 2 337
4 3 180
4 3 337
4 5 -399
4 5 337
4 6 -353
4 6 337
5 1 58
5 1 -399
5 2 -88
5 2 -399
5 3 -180
5 3 -399
5 4 337
5 4 -399
5 6 353
5 6 -399
6 1 -58
6 1 353
6 2 88
6 2 353
6 3 180
6 3 353
6 4 -337
6 4 353
6 5 -399
6 5 353


In [4]:
#Now I'll try to use the determinantal formula to get a kernel vector directly
v = [(-1)^i * det_without_col(A, i) for i in 1:size(A, 2)]
@show v
@show A * v
println("v is in the kernel: ", all(isapprox.(A * v, 0.0, atol=1e-6)))

v = [58, -88, -180, 337, 399, 353]
A * v = 

[0, 0, 0, 0, 0]


v is in the kernel: true


It looks like the determinantal formula given in Sturmfels' book for a kernel
vector in a 1-dimensional kernel really does work. I'm no closer to proving that,
though!

On the bright side, my expansion of the determinants looks correct, as seen in the cell below. Now I just have to figure out what I can do with them...

In [5]:
i = 1
j = 2
println(det_without_col(A, i))
println(det_without_col(A, j))
println()
for k in 1:size(A, 2)
    println(det_without_col_replacing_col(A, i, j, A[:, k]))
end

-58
-88

-88
-58
0
0
0
0


In [9]:
using AbstractAlgebra

num_pairs = binomial(size(A, 2), 2)
dets = zeros(Int, num_pairs, size(A, 2))
current_pair = 0
for i in 1:size(A, 2)
    for j in 1:(i-1)
        if i == j
            continue
        end
        current_pair += 1
        dets[current_pair, i] = det_without_col_replacing_col(A, i, j, A[:, i])
        dets[current_pair, j] = det_without_col_replacing_col(A, i, j, A[:, j])
    end
end
@show dets
#Mdets = matrix(ZZ, dets)
#@show kernel(Mdets)

dets = [-88 -58 0 0 0 0; 180 0 58 0 0 0; 0 180 -88 0 0 0; 337 0 0 -58 0 0; 0 337 0 88 0 0; 0 0 337 180 0 0; -399 0 0 0 58 0; 0 -399 0 0 -88 0; 0 0 -399 0 -180 0; 0 0 0 -399 337 0; 353 0 0 0 0 -58; 0 353 0 0 0 88; 0 0 353 0 0 180; 0 0 0 353 0 -337; 0 0 0 0 353 -399]


15×6 Matrix{Int64}:
  -88   -58     0     0     0     0
  180     0    58     0     0     0
    0   180   -88     0     0     0
  337     0     0   -58     0     0
    0   337     0    88     0     0
    0     0   337   180     0     0
 -399     0     0     0    58     0
    0  -399     0     0   -88     0
    0     0  -399     0  -180     0
    0     0     0  -399   337     0
  353     0     0     0     0   -58
    0   353     0     0     0    88
    0     0   353     0     0   180
    0     0     0   353     0  -337
    0     0     0     0   353  -399

Ok, my idea of building a system of equations with all those dets doesn't look that
bad actually! However, how do I solve it in general?